# Lab 9: Grid Localization using Bayes Filter on the Virtual Robot

#### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

#### This notebook showcases localization results of the **Localization** module (defined in *robot_interface.py*) for the virtual robot.

<hr>


In [3]:
# Automatically reload changes in python modules
%load_ext autoreload
%autoreload 2

# Import classes
from robot_interface import *

import time
import numpy as np
import rospy
from Traj import Trajectory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Instantiate VirtualRobot to communicate with the real robot
robot = VirtualRobot()

# Instantiate Mapper
# Start points for each line segment describing the map
start_points = np.array([[0,0],
                         [5,0],
                         [5,5],
                         [0,5],
                         [0,0],
                         [0,1],
                         [1,1],
                         [1,1.5],
                         [0,1.5],
                         [0,3.5],
                         [2,3.5],
                         [2,2.5],
                         [4.25,2.5],
                         [4.25,1.5],
                         [2.5,0.5]])
    
# End points for each line segment describing the map\n",
end_points = np.array([ [5,0],
                        [5,5],
                        [0,5],
                        [0,0],
                        [0,1],
                        [1,1],
                        [1,1.5],
                        [0,1.5],
                        [0,3.5],
                        [2,3.5],
                        [2,2.5],
                        [4.25,2.5],
                        [4.25,1.5],
                        [2.5,0.5],
                        [0,0]])

# Requires a RealRobot object as input
mapper = Mapper(min_x=0, max_x=5, min_y=0, max_y=5, min_a=-180, max_a=180,
                cell_size_x=0.2, cell_size_y=0.2, cell_size_a=20,
                max_cells_x=25, max_cells_y=25, max_cells_a=18,
                ray_length=7, lines=[start_points, end_points], obs_per_cell=18, 
                robot=robot)
mapper.populate_views()

# Instantiate Localization 
odom_trans_sigma = 0.33
odom_rot_sigma = 15
sensor_sigma = 0.11

# Requires a RealRobot object and a Mapper object as inputs
loc = Localization(robot, mapper, sensor_sigma, odom_trans_sigma, odom_rot_sigma)

# Visualize the map (described using line segments) in the plotter
loc.plotter.visualize_map()

# Bayes filter on the virtual robot
The cell below run the bayes filter algorithm for the pre-planned trajectory.

In [ ]:
# Reset Plots
robot.reset()
loc.plotter.reset_plot()

# Init Uniform Belief
loc.init_pose()

# Get Observation Data by executing a 360 degree rotation motion
#loc.get_observation_data()
loc.obs_range_data = np.array([0.219,0.222,0.251,0.293,0.346,0.335,0.32,0.239,0.262,0.309,0.492,0.439,0.446,0.412,0.267,0.224,0.224,0.317,0.289])
# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
#loc.plotter.plot_point(current_odom[0], current_odom[1], ODOM)
loc.plotter.plot_point(0,3.5,GT)
    
"""
# Intialize the Trajectory object
traj = Trajectory(loc)


# Run through each motion steps
for t in range(0, traj.total_time_steps):

    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
    
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    #loc.get_observation_data()
    loc.obs_range_data = [589.6521739,492.7560976,508.0357143,536.1081081,531.8205128,372.575,273.5555556,249.255814,293.2432432,295.6829268,273.3333333,264.2058824,244.7741935,337.6153846,470.9268293,499.5609756,512.6346154,376.5625]
    
    # Update Step    
    loc.update_step()
    loc.print_update_stats(plot_data=True)
    
        
    print("-------------------------------------")
"""